# CSR、SSR、SSG、ISR

本文主要介绍前端渲染的四种常见方式。文中涉及到 React 和 Next.js 默认指的是版本 18 和 14。

## CSR（Client-side Rendering 客户端渲染）

客户端渲染指的是网页的渲染过程发生在用户的浏览器中。

浏览器从服务器请求一个基础的 HTML 文件，这个文件通常包含一个用于挂载 JS 应用的根 DOM 元素，例如：

```html
<div id="root"></div>
```

React 等 UI 库通过 API 将应用挂载到这个根 DOM 元素上，从而渲染出完整的页面内容。例如：

```tsx
import React from "react";
import ReactDOM from "react-dom/client";

ReactDOM.createRoot(document.getElementById("root")!).render(<App />);
```

### 不足：

**首屏加载慢**

由于页面内容的渲染是在客户端完成的，因此页面所有的 JS 代码块（chunks）都需要从服务器传输到浏览器执行。
较大的 JS 文件会导致加载时间延长，用户可能会看到短暂的白屏或闪烁现象。

**SEO 问题**

传统的搜索引擎爬虫主要依赖静态 HTML 内容进行索引。然而，客户端渲染的内容是通过 JS 动态生成的，这可能会对 SEO 产生影响。

根据 Vercel 团队的最新测试结果，现代搜索引擎（如 Google）能够有效处理和索引 JS 渲染的内容。参考 [How Google handles JavaScript throughout the indexing process](https://vercel.com/blog/how-google-handles-javascript-throughout-the-indexing-process#myth-1-%E2%80%9Cgoogle-can%E2%80%99t-render-javascript-content%E2%80%9D)。

然而，Baidu（百度）在处理 JS 渲染内容方面的支持仍然有限，且目前没有明确的计划进行技术升级。相关信息请参考 [百度搜索引擎工作原理](https://ziyuan.baidu.com/college/courseinfo?id=144)。

拓展资料：

[了解 Google JavaScript SEO](https://developers.google.com/search/docs/crawling-indexing/javascript/javascript-seo-basics?hl=zh-cn)。

### 优势：

**交互流畅**

客户端渲染通过单页应用（SPA）的方式，可以在不刷新页面的情况下更新内容，提供更好的交互体验。

**服务器负担小**

由于渲染工作在客户端进行，服务器只需提供 API 接口返回数据，减少了服务器的计算和渲染压力。这对于高并发的应用尤为重要。

## SSR（Server-side Rendering 服务端渲染）

服务端渲染是指页面的渲染过程发生在服务器端。

与客户端渲染不同，当浏览器向服务器请求 HTML 文件时，服务器会生成一个包含完整内容的 HTML 文档并返回给浏览器。

在服务端渲染中，浏览器接收到的 HTML 文档已经包含了完整的 DOM 结构，但在初始加载时，这个页面是静态的，暂时不具备交互能力。

随后，React 等 UI 库会在浏览器端对这个 HTML 进行水合（Hydration）操作，将其转换为“活跃”的页面，使其具备交互功能。

只有在水合完成后，用户才能与页面进行交互。过程如下：

```tsx
import { hydrateRoot } from "react-dom/client";

hydrateRoot(document.getElementById("root"), <App />);
```

水合是 SSR 特有的过程，而在 CSR 中，由于页面内容完全由客户端生成，因此不存在水合的概念。

## SSG（Static Site Generation 静态站点生成）

静态站点生成是指页面的渲染过程发生在项目构建时，而不是在客户端或服务器端的运行时。

在开发者运行构建命令时（如 Next.js 的 `next build`），SSG 工具会在服务器端或本地环境预先渲染所有页面，
将每个页面生成一个静态的 HTML 文件。这些 HTML 文件包含了预渲染的内容和样式，类似于 SSR 的结果，但这个渲染过程仅发生一次（即构建时）。

生成的静态 HTML 文件会被部署到静态文件服务器或 CDN 上。当用户访问网站时，服务器直接返回这些预生成的 HTML 文件，而无需在运行时进行任何额外的渲染操作。

## ISR（Incremental Static Regeneration 增量静态再生）

增量静态再生（ISR）结合了静态站点生成（SSG）和服务端渲染（SSR）的特点。

在初次构建时，ISR 会像 SSG 一样预生成静态页面，并将其部署到服务器或 CDN。

当用户访问页面时，如果页面已经过期，ISR 会触发一个后台再生成过程。这个过程在服务器端完成，类似于 SSR，但与 SSR 不同的是，再生成的页面会被存储为静态文件，而不是每次请求都进行实时渲染。

具体来说，再生成的流程如下：

1. 用户请求过期的页面时，ISR 会启动一个后台再生成过程。
2. 服务器重新生成页面，并将新的 HTML 文件更新到缓存中。
3. 在生成过程完成后，下一个用户请求将会获得更新后的页面。

举个例子，假设一个 Next.js 页面设置了 `revalidate` 选项为 10 秒，这意味着页面在构建时将会生成静态 HTML 文件，并在 10 秒内保持不变。

具体流程如下：

1. **0-10 秒**：在初始构建完成后的前 10 秒内，用户请求的 HTML 文件都是之前生成的静态页面。页面不会重新生成，所有请求都直接返回缓存的静态页面。

2. **第 11 秒**：当第一个请求到达并发现页面已经过期时（超过了 10 秒），ISR 会启动一个后台再生成过程。此时，服务器仍然会返回旧的缓存页面给用户，同时在后台生成新的页面。

3. **生成完成后**：当后台生成的页面完成后，下一个用户请求将会得到新的页面。新生成的页面会被存储为缓存静态文件，并在后续的 10 秒内继续使用，直到再次触发再生成过程。

这种机制保证了页面在大部分时间内快速响应，同时能够在后台静默地更新内容，确保用户能尽快看到最新的数据。


总结：

| 渲染方式            | 渲染时机      | 渲染过程描述                                                                                                                   | 网络传输情况                   | 优点                                     | 缺点                                     |
| ------------------- | ------------- | ------------------------------------------------------------------------------------------------------------------------------ | ------------------------------ | ---------------------------------------- | ---------------------------------------- |
| 客户端渲染（CSR）   | 运行时        | 使用 JS 在浏览器端根据根 DOM 元素生成页面。浏览器需要完整的组件 JS chunks 来动态渲染页面。                                     | 网络传输的 JS 文件较多         | 动态更新页面，适合高度交互的应用。       | 初次加载慢，SEO 支持差，首屏加载不友好。 |
| 服务端渲染（SSR）   | 运行时        | 在服务器端根据数据生成完整的 DOM 结构，并将其发给浏览器。浏览器端通过 JS 对静态 HTML 进行水合（Hydration），使其具备交互能力。 | 网络传输的 JS 文件较少         | 首屏加载快，SEO 友好。                   | 服务器负担重，动态内容处理复杂。         |
| 静态站点生成（SSG） | 构建时        | 在构建过程中生成静态 HTML 页面，这些静态页面可以在浏览器中直接展示，无需进一步渲染。                                           | 网络传输的静态 HTML 文件较少   | 页面加载极快，服务器负担轻。             | 构建时间长，内容更新不即时。             |
| 增量静态再生（ISR） | 构建时+运行时 | 在构建时生成静态 HTML 页面供初次加载，页面过期后在后台重新生成页面供后续访问。                                                 | 结合了静态文件和动态更新的优点 | 结合了 SSG 和 SSR 的优点，灵活更新内容。 | 缓存管理                                 |
